In [ ]:
def histogramify(image):
    histogram = np.empty((256))
    for grays in range(0, 256):
        histogram[grays] = len(image[image == grays])

    return histogram

def aveFiltersApplied(img, kernels):
    vals = np.array([signal.convolve2d(img, kernel, 'same') for kernel in kernels])
    return vals

def lexiSort(matrix):
    rotmatrix = np.rot90(matrix)
    index = np.lexsort(rotmatrix)
    return matrix[index]

# Discret Fourier Transfrom
def DFT(image):
    return np.fft.fftshift(np.fft.fft2(image))

# inverse DFT
def iDFT(image):
    return np.fft.ifft2(image)

# adds padding around an image using the edge mode
def padding(image):
    p = np.pad(image, 1, mode='edge')
    return p

#generates a notch filters
def notchFilter(shape, D0, uk, vk):
    M, N = shape
    Huv = np.zeros((M, N))

    for u in range(0, M):
        for v in range(0, N):
            D_vek = np.sqrt((u-M/2+uk)**2 + (v-N/2+vk)**2)
            D_k = np.sqrt((u-M/2-uk)**2 + (v-N/2-vk)**2)
            Huv[u,v] += ((1/(1+(D0/D_k)**4))*(1/(1+(D0/D_vek)**4))) 

    return Huv

def avePixels(imgage, K):
    kernels = aveFilters[K]
    aveImages = aveFiltersApplied(imgage, kernels)
    img_size = aveImages[0].shape[0] * aveImages[0].shape[1]
    reshapedAveImages = aveImages.reshape((K, img_size))
    transposedAveImages = reshapedAveImages.transpose()
    return transposedAveImages

def BRB(image,D0,W,n):
    paddedImaged = padding(image)
    m = paddedImaged.shape[0]
    n = paddedImaged.shape[1]
    Huv = np.zeros((m, n))
    
    for u in range(0,m):
        for v in range(0,n):
            D = np.sqrt((u-m/2)**2+(v-n/2)**2)
            Huv[u,v]+=(1/(1+(D*W/(D**2-D0**2)**(2*n))))
    
    return Huv

#applies the BRB filter generated as a result of the above functions
def applyBRB(image, brb):
    p = padding(image)
    dft = DFT(p)
    G = np.multiply(brb,dft)
    Gi = iDFT(G)
    S = np.log(1+abs(Gi))
    return S

def exact_histogram_matching(image, im_target, K):
    refHistogram = histogramify(im_target)
    size = image.shape[0] * image.shape[1]
    merged = np.empty((size, K + 2))
    merged[:, 0] = image.reshape((size,))
    flattenedImage = np.arange(size).transpose()
    merged[:, -1] = flattenedImage
    aveImages = avePixels(image, K)
    
    for dimension in range(0, K):
        merged[:, dimension + 1] = aveImages[:, dimension]

    sortedMerged = lexiSort(merged)
    start = 0
    
    for gray_value in range(0, len(refHistogram)):
        end = int(start + refHistogram[gray_value])
        sortedMerged[start:end, 0] = gray_value
        start = end

    sortedMerged = sortedMerged[sortedMerged[:, -1].argsort()]
    result = sortedMerged[:, 0].reshape(image.shape)

    return result

# Generates the cumulative distribution function of an image
def getCDF(image):
    hist,bins = np.histogram(image.flatten(),256,[0,256])
    cdf = hist.cumsum()
    cdf_norm = cdf * hist.max()/ cdf.max()
    
    return cdf_norm

fshift = DFT(car)
oFourier = np.log(np.abs(1+fshift))
shape = car.shape

N1 = notchFilter(shape, 9, 38, 30) #Notch 1
N2 = notchFilter(shape, 9, -42, 27) #Notch 2
N3 = notchFilter(shape, 9, 80, 30) #Notch 3
N4 = notchFilter(shape, 9, -82, 28) #Notch 4

Filter = N1*N2*N3*N4
void = fshift * Filter 
reject = np.fft.ifftshift(void)
ireject = iDFT(reject)
output = np.abs(ireject)
###################################################### 2 ######################################################
def histogramify(image,mode):
    # array with size of bins, set to zeros
    histogram = np.zeros(256)
    
    # loop through pixels and sum up counts of pixels
    for pixel in image:
        histogram[pixel] += 1
    
    # return our final result
    return histogram

def cumsum(a):
    a = iter(a)
    b = [next(a)]
    for i in a:
        b.append(b[-1] + i)
    return np.array(b)

def histEqual(image):
    h = histogramify(image,1)
    cum = cumsum(h)
    nj = (cum - cum.min()) * 255
    N = cum.max() - cum.min()
    cum = nj / N
    cum = cum.astype('uint8')
    img_new = cum[image]
    img_new = np.reshape(img_new, image.shape)
    return img_new

lst = []
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
         lst.append(np.binary_repr(img[i][j] ,width=8)) # width = no. of bits

# We have a list of strings where each string represents binary pixel value. To extract bit planes we need to iterate over the strings and store the characters corresponding to bit planes into lists.
# Multiply with 2^(n-1) and reshape to reconstruct the bit image.
eight_bit_img = (np.array([int(i[0]) for i in lst],dtype = np.uint8) * 128).reshape(img.shape[0],img.shape[1])
seven_bit_img = (np.array([int(i[1]) for i in lst],dtype = np.uint8) * 64).reshape(img.shape[0],img.shape[1])
six_bit_img = (np.array([int(i[2]) for i in lst],dtype = np.uint8) * 32).reshape(img.shape[0],img.shape[1])
five_bit_img = (np.array([int(i[3]) for i in lst],dtype = np.uint8) * 16).reshape(img.shape[0],img.shape[1])
four_bit_img = (np.array([int(i[4]) for i in lst],dtype = np.uint8) * 8).reshape(img.shape[0],img.shape[1])
three_bit_img = (np.array([int(i[5]) for i in lst],dtype = np.uint8) * 4).reshape(img.shape[0],img.shape[1])
two_bit_img = (np.array([int(i[6]) for i in lst],dtype = np.uint8) * 2).reshape(img.shape[0],img.shape[1])
one_bit_img = (np.array([int(i[7]) for i in lst],dtype = np.uint8) * 1).reshape(img.shape[0],img.shape[1])

imshow(one_bit_img,cmap='gray')

def print_histogram(_histrogram, name, title):
    plt.figure()
    plt.title(title)
    plt.plot(_histrogram, color='#ef476f')
    plt.bar(np.arange(len(_histrogram)), _histrogram, color='#b7b7a4')
    plt.ylabel('Number of Pixels')
    plt.xlabel('Pixel Value')
    plt.savefig("hist_" + name)


def generate_histogram(img):
    gr_hist = np.zeros([256])
    gr_img = img

    for x_pixel in range(gr_img.shape[0]):
        for y_pixel in range(gr_img.shape[1]):
            pixel_value = int(gr_img[x_pixel, y_pixel])
            gr_hist[pixel_value] += 1
    '''normalize Histogram'''
    #gr_hist /= (gr_img.shape[0] * gr_img.shape[1])


def equalize_histogram(img, histo, L):
    eq_histo = np.zeros_like(histo)
    en_img = np.zeros_like(img)
    for i in range(len(histo)):
        eq_histo[i] = int((L - 1) * np.sum(histo[0:i]))
    print_histogram(eq_histo, name="eq_"+str(index), title="Equalized Histogram")
    '''enhance image as well:'''
    for x_pixel in range(img.shape[0]):
        for y_pixel in range(img.shape[1]):
            pixel_val = int(img[x_pixel, y_pixel])
            en_img[x_pixel, y_pixel] = eq_histo[pixel_val]
    '''creating new histogram'''
    hist_img, _ = generate_histogram(en_img, print=False, index=index)
    return eq_histo


def find_value_target(val, target_arr):
    key = np.where(target_arr == val)[0]

    if len(key) == 0:
        key = find_value_target(val+1, target_arr)
        if len(key) == 0:
            key = find_value_target(val-1, target_arr)
    vvv = key[0]
    return vvv


def match_histogram(inp_img, hist_input, e_hist_input, e_hist_target):
    '''map from e_inp_hist to 'target_hist '''
    en_img = np.zeros_like(inp_img)
    tran_hist = np.zeros_like(e_hist_input)
    for i in range(len(e_hist_input)):
        tran_hist[i] = find_value_target(val=e_hist_input[i], target_arr=e_hist_target)
    print_histogram(tran_hist, name="trans_hist_", title="Transferred Histogram")
    '''enhance image as well:'''
    for x_pixel in range(inp_img.shape[0]):
        for y_pixel in range(inp_img.shape[1]):
            pixel_val = int(inp_img[x_pixel, y_pixel])
            en_img[x_pixel, y_pixel] = tran_hist[pixel_val]
    '''creating new histogram'''
    hist_img = generate_histogram(en_img, print=False, index=3)
    return en_img

###################################################### 3 ######################################################

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.144])

def convolve2D(image, kernel, padding=0, strides=1):
    # Cross Correlation

    kernel = np.flipud(np.fliplr(kernel))


    # Gather Shapes of Kernel + Image + Padding
    xKernShape = kernel.shape[0]
    yKernShape = kernel.shape[1]
    xImgShape = image.shape[0]
    yImgShape = image.shape[1]

    # Shape of Output Convolution
    xOutput = int(((xImgShape - xKernShape + 2 * padding) / strides) + 1)
    yOutput = int(((yImgShape - yKernShape + 2 * padding) / strides) + 1)
    output = np.zeros((xOutput, yOutput))

    # Apply Equal Padding to All Sides
    if padding != 0:
        imagePadded = np.zeros((image.shape[0] + padding*2, image.shape[1] + padding*2))
        imagePadded[int(padding):int(-1 * padding), int(padding):int(-1 * padding)] = image
        print(imagePadded)
    else:
        imagePadded = image

    # Iterate through image
    for y in range(image.shape[1]):
        # Exit Convolution
        if y > image.shape[1] - yKernShape:
            break
        # Only Convolve if y has gone down by the specified Strides
        if y % strides == 0:
            for x in range(image.shape[0]):
                # Go to next row once kernel is out of bounds
                if x > image.shape[0] - xKernShape:
                    break
                try:
                    # Only Convolve if x has moved by the specified Strides
                    if x % strides == 0:
                        output[x, y] = (kernel * imagePadded[x: x + xKernShape, y: y + yKernShape]).sum()
                except:
                    break

    return output

def gaussian2DFilter(size,sigma):
    gausskernel = np.zeros((size,size),np.float32)
    for i in range (size):
        for j in range (size):
            norm = math.pow(i-1,2) + math.pow(j-1,2)
            gausskernel[i,j] = math.exp(-norm/(2*math.pow(sigma,2)))/2*math.pi*pow(sigma,2)
    sum = np.sum(gausskernel)
    kernel = gausskernel/sum
    return kernel

###################################################### 4 ######################################################
def myconvolve2(f, g) :                                                                                                                                                           
    # Pad g to equal size of f. This assumes f is smaller in both dimensions                                                                                                      
    # and that the difference between f and g dimensions are even                                                                                                                 
    p1, p2 = (np.r_[f.shape]-g.shape).astype(int)//2                                                                                                                              
    gpad = np.pad(g,((p1,p1),(p2,p2)),mode='edge')
    
    gpadded = np.zeros((f.shape[0], f.shape[1]))
    gpadded[int(1):int(-1 * 1), int(1):int(-1 * 1)] = g
    print(gpadded)

    # Shift g to 'center' on top left corner (the 'origin' in an fft)                                                                                                             
    gpad = np.fft.ifftshift(gpad)                                                                                                                                                  

    # Multiply spectra                                                                                                                                                            
    FG = np.fft.fft2(f) * np.conj(np.fft.fft2(gpad))                                                                                                                              
    return np.real(np.fft.ifft2(FG)), gpad 

###################################################### 6 ######################################################
def binarize_this(image_file, thresh_val=127):
    image_src = image_file
    image_b = convert_binary(image_src=image_src, thresh_val=thresh_val)
    return image_b

def convert_binary(image_src, thresh_val):
    color_1 = 255
    color_2 = 0
    initial_conv = np.where((image_src <= thresh_val), image_src, color_1)
    final_conv = np.where((initial_conv > thresh_val), initial_conv, color_2)
    return final_conv

def erode_this(image_file, erosion_level=1, with_plot=False):
    erosion_level = 3 if erosion_level < 3 else erosion_level

    structuring_kernel = np.full(shape=(erosion_level, erosion_level), fill_value=255)
    image_src = binarize_this(image_file=image_file)

    orig_shape = image_src.shape
    pad_width = erosion_level - 2

    # pad the matrix with `pad_width`
    image_pad = np.pad(array=image_src, pad_width=pad_width, mode='constant')
    pimg_shape = image_pad.shape
    h_reduce, w_reduce = (pimg_shape[0] - orig_shape[0]), (pimg_shape[1] - orig_shape[1])

    # sub matrices of kernel size
    flat_submatrices = np.array([
        image_pad[i:(i + erosion_level), j:(j + erosion_level)]
        for i in range(pimg_shape[0] - h_reduce) for j in range(pimg_shape[1] - w_reduce)
    ])

    # condition to replace the values - if the kernel equal to submatrix then 255 else 0
    image_erode = np.array([255 if (i == structuring_kernel).all() else 0 for i in flat_submatrices])
    image_erode = image_erode.reshape(orig_shape)

    if with_plot:
        cmap_val = 'gray'
        fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 20))

        ax1.axis("off")
        ax1.title.set_text('Original')

        ax2.axis("off")
        ax2.title.set_text("Eroded - {}".format(erosion_level))

        ax1.imshow(image_src, cmap=cmap_val)
        ax2.imshow(image_erode, cmap=cmap_val)
        plt.show()
        return True
    return image_erode

fig, (cx1,cx2) = plt.subplots(nrows=1, ncols= 2, figsize=(8,3), sharex=True, sharey=True)
for bb in (cx1,cx2):
    bb.set_axis_off()

cx2.imshow(dw1, cmap = 'gray')
cx1.imshow(originalFourier*b1, cmap = 'gray')